# <span style="color:lightcoral; font-family:freestyle script; font-size:4em">SLOSH till relationsdatabas</span> <br><span style="color:black; font-family:freestyle script; font-size:3em">Förbereda färdig metadata för SQLite</span>
***

# Innehållsförteckning  
1. [Introduktion](#1)  
2. [Kolumner](#2)  
    2.1 [Enkät](#2.1)  
    2.2 [Itemnr](#2.2)  
    2.3 [Frekvens](#2.3)
3. [Tabeller](#3)  
    3.1 [Variabler](#3.1)  
    3.2 [Frekvenstabell](#3.2)  
    3.3 [Enkättexter](#3.3)  
    3.4 [Kodlistor](#3.4)  
    3.5 [Svarstexter](#3.5)  
    3.6 [Kodlistor_stor](#3.6)

***

# <a id = "1">1. Introduktion</a>  
I vad som följer kommer kursiv stil endast användas för att benämna *tabeller*. Fetstil kommer endast användas för att benämna **kolumner**, och värden i en kolumn kommer att vara i stil med (t.ex.) `SLOSHW08`, `Missing`, `142`. Även datatyper (t.ex. `str` eller `float64`) och pythonobjekt (t.ex. listor) kommer att ha denna formatering. Den metadata som redan är dokumenterad enligt Vetenskapsrådets riktlinjer ligger i excelfilen *metadata*. Några av tabellerna i relationsdatabsen innehåller kolumner som inte finns i denna fil. Dessa kolumner skapas i avsnittet [Kolumner](#2). Därefter kommer följande tabeller att skapas:  

|Variabler|Frekvenstabell|Enkättexter|Kodlistor|Svarstexter|Kodlistor_stor|  
|:---:|:---:|---|---|---|---|
|Variabel|V_id|Enkättext|Kodlista|Svarstext|K_id|  
|Enkät|Variabel|Källa|From|From|Kodlista|  
|Beskrivning|Enkät|-|Tom|Tom|Kod|  
|Itemnr|Kod|-|-|-|Svarstext|  
|Enkättext|Frekvens|-|-|-|-|  
|Kodlista|-|-|-|-|-|  
  
  
Relationsdatabasen kommmer att innehålla ytterligare fyra tabeller. Dessa är *Begrepp*, *Begrepp_stor*, *Tidsserie* samt *Tidsserie_stor*. För tillfället finns ingen data till dessa tabeller i *metadata* och därför kommer de inte skapas här. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('Metadata.xlsx')

För att ta fram *Kodlista_stor* används `missing` i **text**. I *metadata* stavas `Missing` ibland med "M" och ibland med "m"(`missing`). Python läser detta som olika `str`, därför ersätts liten bokstav med stor.

In [3]:
df.replace('missing', 'Missing', inplace = True)

In [4]:
df.columns

Index(['Namn', 'Tidsserie', 'Beskrivning', 'Enkät', 'Dubbelkodning',
       'Enkättext', 'From', 'Tom', 'Kodlista', 'From.1', 'Tom.1', 'Kod',
       'Text', 'From.2', 'Tom.2', 'Koncept 1', 'Koncept 2', 'Instrument 1',
       'Instrument 2', 'Källa', 'SLOSHW06', 'frek', 'SLOSHW08', 'frek.1',
       'SLOSHW10', 'frek.2', 'SLOSHW12', 'frek.3', 'SLOSHW14', 'frek.4',
       'SLOSHW16', 'frek.5', 'SLOSHW18', 'frek.6', 'SLOSHNW06', 'frek.7',
       'SLOSHNW08', 'frek.8', 'SLOSHNW10', 'frek.9', 'SLOSHNW12', 'frek.10',
       'SLOSHNW14', 'frek.11', 'SLOSHNW16', 'frek.12', 'SLOSHNW18', 'frek.13'],
      dtype='object')

*** 
# <a id = "2">2. Ny kolumn: Itemnr</a>  
Det måste skapas en ny kolumn som kallas **Itemnr**.  

**Itemnr** ska innehålla `str` som för närvarande ligger utspridda i vågkolumnerna i *metadata*. Ett värde i kolumnen kan t.ex. vara `A26_b`. Här är ett förenklat schema som visar relationen mellan **Itemnr** och vågkolumnerna i *metadata*:

In [ ]:
pd.DataFrame([['A25', 'NaN', 'Nan', 'A25'], ['NaN', 'B23', 'Nan', 'B23'], ['NaN', 'NaN', 'C13', 'C13']], 
             columns = ['SLOSHW06', 'SLOSHNW10', 'SLOSHW16', 'Itemnr'])

Först skapas en ny kolumn som får namnet "Itemnr", kolumnen fylls med `NaN`.

In [ ]:
df['Itemnr'] = np.nan

Proceduren för att välja ut rätt itemnummer illustreras i exemplet nedan.  
***  
  <span style="color:red">**Exempel**

In [ ]:
d = pd.DataFrame([[1,2,3], [4,5,6], [1,8,9]], columns = ['A', 'B', 'C'])
d['test'] = 0

In [ ]:
d

In [ ]:
d.loc[d['A'] == 1, ['test']] = d['B']
d

<span style="color:red">**Exempel slut**  
***

Loopen nedan går igenom vågkolumnerna i *metadata*. Om en variabel har fått värdet `SLOSHW06` i **Enkät** kommer värdet för samma variabel i vågkolumnen **SLOSHW06** att bli variabelns värde i den nya kolumnen **Itemnr**.

In [ ]:
for w, nw in zip(WORKING, NONWORKING):
    df.loc[df['Enkät'] == w, ['Itemnr']] = df[w]
    df.loc[df['Enkät'] == nw, ['Itemnr']] = df[nw]

***  
# <a id = "3">3. Tabeller</a>  
I det här avnittet kommer tabellerna (förutom de som rör tidsserier och begrepp, se [introduktionen](#1)) till relationsdatabasen att skapas. Först tabellen *Variabler* (som är störst). Därefter *Frekvenstabeller*, *Enkättexter*, *Kodlistor*, *Svarstexter* och till sist *Kodlistor_stor*. 

***  
## <a id = "3.1">3.1 Variabler</a>  
Den här tabellen ska innehålla kolumnerna **Variabel**, **Enkät**, **Beskrivning**, **Itemnr**, **Enkättext** och **Kodlista**. Samtliga kolumner finns redan i *metadata* (numera `df`), dock kallas **Variabel** där för **Namn** och måste därför döpas om. 

In [ ]:
Variabler = df[['Namn', 'Enkät', 'Beskrivning', 'Itemnr', 'Enkättext', 'Kodlista']]
Variabler = Variabler.rename({'Namn': 'Variabel'}, axis = 'columns')

Värden i **Variabel** ska vara unika.

In [ ]:
Variabler.drop_duplicates(['Variabel'], inplace = True)

<span style="color: red"> **här skapas kolumnen med enkätvärden med hjälp av slosh-datafilen**</span>

***  
## <a id = "3.2">3.2 Frekvenstabeller</a>  
Kolumnerna är **V_id**, **Variabel**, **Enkät**, **Kod** och **Frekvens**.

In [ ]:
Frekvenstabeller = df[['Namn', 'Enkät', 'Kod', 'Frekvens']]
Frekvenstabeller = Frekvenstabeller.rename({'Namn': 'Variabel'}, axis = 'columns')

**V_id** är huvudnyckel i tabellen, därför räcker det att döpa om index.

In [ ]:
Frekvenstabeller.index.name = 'V_id'

***  
## <a id = "3.3">3.3 Enkättexter</a>  
Består endast av **Enkättext** och **Källa**. Eventuellt bör **Enkättext** göras till index eftersom den är huvudnyckel i tabellen.

In [ ]:
Enkättexter = df[['Enkättext', 'Källa']]
Enkättexter.set_index('Enkättext', inplace = True)

***  
## <a id = "3.4">3.4 Kodlistor</a>  
Består av **Kodlista** (nyckel) samt **From.1** och **Tom.1**. Notera att de sista klolumnerna blivit omdöpta automatiskt vid inläsning av *metadata*. 

In [ ]:
Kodlistor = df[['Kodlista', 'From.1', 'Tom.1']]
Kodlistor.set_index('Kodlista', inplace = True)

***  
## <a id = "3.5">3.5 Svarstexter</a>  
Består av **Svarstext** (nyckel) samt **From** och **Tom**. Den första kolumnen heter **Text** i *metadata* och döps därför om. 

In [ ]:
Svarstexter = df[['Text', 'From', 'Tom']]
Svarstexter.rename(columns = {'Text':'Svarstext'}, inplace = True)
Svarstexter.set_index('Svarstext', inplace = True)

***  
## <a id = "3.6">3.6 Kodlistor_stor</a>  
Den här tabellen ska innehålla kodlistornas namn i kombination med deras kod och svarstext. Varje sådan kombination ska endast förekomma en gång. Börja med att skära ut kolumnerna **Kodlista**, **Kod** och **Text** ur *metadata*.

In [ ]:
d = df[['Kodlista', 'Kod', 'Text']]  
d.head(5)

I den här mindre tabellen förekommer vissa kodlistor flera gånger. För att identifiera dubletter behövs en slags räknare. Börja med att lägga till en ny kolumn fylld med `0`. Kalla kolumnen för **K_id**.

In [ ]:
d.loc[:, 'K_id'] = 0
d.head(5)

**K_id** ska vara unik för varje kombination. Loopen nedan ökar **K_id** med `1` varje gång den passerar en kombination. En kombination identifieras genom att dess sista rad alltid har svarstexten `Missing`.

In [ ]:
for id in range(1, len(d['K_id'])):
    if d.iloc[id-1, 2] == 'Missing':
        d.iloc[id: , 3] = d.iloc[id: , 3] + 1

Loopen nedan lägger varje kodlista i listan `behåll` endast en gång. Varje gång en kodlista läggs i `behåll` lägger loopen tillhörande **K_id** i listan `unikid`. Denna andra lista innehåller alltså alla unika förekomster av kombinationer.

In [ ]:
unikid = []
behåll = []
for id in range(len(d['K_id'])):
    if d.iloc[id, 0] not in behåll:
        unikid.append(d.iloc[id, 3])
        behåll.append(d.iloc[id, 0])
        

Tills sist skapas tabellen *Kodlistor_stor* genom att endast ha med de rader för vilka **K_id** återfinns i listan `unikid`.

In [ ]:
Kodlistor_stor = d.loc[d['K_id'].isin(unikid)]

Gör om **K_id** till ett index så att den kan agera nyckel i tabellen. 

In [ ]:
Kodlistor_stor.loc[:, 'K_id'] = Kodlistor_stor.index
Kodlistor_stor.set_index('K_id', inplace = True)

***  
<span style="color:dodgerblue; font-family:freestyle script; font-size:3em">SKISS</span>

Variabler för vilka enkät = `w+n` och dubbelkodning = `Nan` har två frekvenstabeller efter varandra i radledd. I det här avsnittet kommer sådana frekvenstabeller, som alltså ligger till höger om (i kolumnledd) variabeln ifråga, att flyttas ner. Variabeln kommer att klippas ut och infogas direkt under sig själv. I det övre läget kommer endast frekventabell för arbetande att förekomma, och i det undre ska frekvenstabellen för icke-arbetande ligga.  

### Förkortningar
* $ V = $ kombination av rader med samma variabelnamn samt **Enkät** $ = $ `w+n` och **Dubbelkodning** $ = $ `NaN`
* $ v = $ första raden i något $ V $  
* $D = V + D'$  
* $D' = D - V$  
* $d = $ första raden i något $ D $  
* $DV = D + V$

Läs in *metadata* i en ny `DataFrame` eftersom **Enkät** har blivit ändrad. Identifiera radnummer för de variabler som har **Enkät** = `w+n` samt **Dubbelkodning** = `NaN`. Varje kombination av rader som uppfyller detta villkor och som har samma variabelnamn kallas för $V$.

In [6]:
df1 = pd.read_excel('Metadata.xlsx')

df2 = df1[(df1.Enkät == 'w+n') & (df1.Dubbelkodning.isna() == True)]

Notera att avståndet mellan index för dessa rader inte längre måste vara 1 (som i ett vanligt index). Skapa därför en ny kolumn och lägg index till de utvalda raderna där, det blir enklare att loopa över ett "vanligt" index (0, 1, 2, 3, 4, ...) senare.

In [7]:
df2['id'] = df2.index

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Välj ut det första radnummret $v$ för varje $V$. I den här loopen refererar kolumn 0 till **Namn** och -1 till **id**.

In [8]:
namn = []
v = []

for i in df2.index:
    if df2.loc[i, 'Namn'] not in namn:
        namn.append(df2.loc[i, 'Namn'])
        v.append(df2.loc[i, 'id'])

Låt $n$ vara antalet $V$. Tabellen *metadata* måste delas upp i $n$ delar ($D$). Varje $D$ består av $V$ samt $D' = D-V$. Varje $D$ har en första rad som kallas $d$. Ifall två $V$ följer efter varandra i *metadata* kommer $D'$ vara tomt. För vissa $D$ kommer alltså $d$ och $v$ sammanfalla.  
Den första $d$ i *metadata* är såklart `0`. Välj ut resterande $d$ genom att hitta den första rad efter $v$ i vilken variabelnamnet ändras.  

In [9]:
d = [0]

for i in v:
    for j in range(i+1, len(df1.loc[:, 'Namn'])):
        if df1.loc[j, 'Namn'] != df1.loc[j-1, 'Namn']:
            d.append(j)
            break

Nu finns en lista med samtliga $d$. Skapa en lista `e` som är precis som `d` fast utan det första elementet och med ett extra element i slutet. varje par $(d, e)$ utgör nu första respektive sista rad i något $D$.

In [10]:
e = d[1:]
e.append(len(df1.iloc[:, 0]))

Skär ut samtliga $D$ och lägg i en lista.

In [11]:
D = [df1.iloc[i:j, :] for i, j in zip(d, e)]

Gå igenom samtliga $D$ och ersätt samtliga värden i frekvenstabellerna i listan `NONWORKFREK` med `NaN`.

In [12]:
for i, j in zip(D, v):
    for k in NONWORKFREK:
        i.loc[j:, k] = np.nan        

NameError: name 'NONWORKFREK' is not defined

På samma sätt utgör $(v, e)$ första respektive sista rad i $V$. Skär ut samtliga $V$ och lägg i en lista.

In [ ]:
V = [df1.iloc[i:j, :] for i, j in zip(v, e)]

Gå igenom samtliga $V$ och ersätt samtliga värden i frekvenstabellerna listan `WORKFREK` med `NaN`.

In [ ]:
for i in V:
    for j in WORKFREK:
        i.loc[:, j] = np.nan

Nu ska varje $D$ sammanfogas i radledd med sitt tillhörande $V$. Spara resultatet i listan `DV` .

In [ ]:
DV = [pd.concat([i, j]) for i, j in zip(D, V)]

Notera att det verkar finnas, längst ner i *metadata*, ett stycke $D$ som inte har något $V$. Detta går att se genom att jämföra längden på listan `D` med listan `V`. Listan `D` har ett element mer än `V`. Eftersom listorna är av olika längd kommer det sista elementet i `D` att undantas från sammanslagning i föregående rad. Lägg därför detta element från `D` längst bak i `DV`.

In [ ]:
DV.append(D[-1])

Sammanfonga nu samtliga element i `DV` i radledd och skapa ett nytt index.

In [ ]:
df3 = pd.concat(DV, ignore_index = True)

In [ ]:
type(df3.loc[0, 'Dubbelkodning']) == 'nan'

In [ ]:
df3[(df3.Dubbelkodning.isna() == True) & (df3.Enkät == 'w+n')].to_excel('test.xlsx')